# Оценка использования основных фондов (ОФ) предприятия

## Исходные данные (вариант 10)

### Таблица 1

* Среднегодовая балансовая стоимость ОПФ (Фо): тыс. у.д.е.
* Средние фактические сроки службы ОПФ (Тф): лет         

In [ ]:
import pandas as pd

table1 = pd.DataFrame(data={
    'Фо': [19450, 1633, 945, 1084, 31200, 1740, 2232, 1212, 1016, 470],
    'Тф': [28, 16, 11, 12, 6, 5, 4, 6, 3, 8]
}, index=[
    'Здания',
    'Сооружения',
    'Передаточные устройства',
    'Силовые машины и оборудование',
    'Рабочие машины и оборудование',
    'Измерительные и регулирующие приборы и устройства, лабораторн. обор-е',
    'Вычислительная техника',
    'Транспортные средства',
    'Инструмент',
    'Производственный и хозяйств. инвентарь и принадлежности'
])
table1.append(table1.select_dtypes(pd.np.number).sum().rename('ВСЕГО ОПФ'))

In [ ]:
table2 = pd.DataFrame(data={'Показатели': [17.4, 1994, 4, 12]}, index=[
    'Оптовая цена станка, тыс.руб.',
    'Год приобретения',
    'Транспортные расходы, % от оптовой цены',
    'Затраты на монтаж и отладку, % от оптовой цены'
])
# table2['Показатели']['Оптовая цена станка, тыс.руб.']
table2

In [ ]:
table3 = pd.DataFrame(data={'Показатели': [48, 12, 8, 7, 7]}, index=[
    'Стоимость ОПФ на 1 января текущего года, тыс. у.д.е.',
    'Введено ОПФ в текущем году, тыс. у.д.е.',
    'Дата ввода, номер месяца',
    'Выбыло ОПФ в текущем году, тыс. у.д.е.',
    'Дата выбытия, номер месяца'
])
table3

In [ ]:
table4 = pd.DataFrame(data={
    'План': [132, 55, 24],
    'Отчет': [133.1, 52.2, 26]
}, index=[
    'Валовая продукция, тыс. у.д.е.',
    'Среднегодовая стоимость основных производственных фондов, тыс. у.д.е.',
    'Среднегодовая численность производственных рабочих, чел.',
])
table4

In [ ]:
table5 = pd.DataFrame(data={'Показатели': [25, 10, 1520, 7568]}, index=[
    'Количество станков в цехе',
    'Средняя часовая производительность одного станка, шт.',
    'Фактический выпуск изделий за смену, шт.',
    'Фактически отработано за месяц, станко-часов'
])
table5

## Определение структуры и состава основных фондов

### 1. Удельный вес $d$ отдельных групп ОФ в общей их стоимости

$$d = \frac{\overline{Ф}_{oi}}{\sum^n_{i=1} \overline{Ф}_{oi}}$$

где $\overline{Ф}_{oi}$ — среднегодовая стоимость $i$-й группы ОФ, $n$ — число групп.

In [ ]:
pd.DataFrame((table1['Фо'] / table1['Фо'].sum()).rename('Удельный вес $d$'))

### 2. Удельный вес активной части в общей стоимости ОФ

$$d_a = \frac{\sum^m_{i=1} \overline{Ф}_{oa}}{\sum^n_{i=1} \overline{Ф}_{oi}}$$

где $\overline{Ф}_{oi}$ — среднегодовая стоимость $i$-й группы активной части ОФ, $m$ — число групп в составе активной части ОФ (машины и оборудование, инструмент и др.)

In [ ]:
import numpy as np

np.round(table1['Фо'][2:].sum() / table1['Фо'].sum(), 4)

### 3. Удельный вес отдельных возрастных групп

$$d_j = \frac{\sum^k_{i=1} \overline{Ф}_{oi}}{\sum^n_{i=1} \overline{Ф}_{oi}}$$

где $k$ — число групп в составе $j$-й возрастной группы

In [ ]:
pd.DataFrame(data={'Удельный вес': [
    table1.query('Тф < 5')['Фо'].sum() / table1['Фо'].sum(),
    table1.query('Тф >= 5 & Тф < 10')['Фо'].sum() / table1['Фо'].sum(),
    table1.query('Тф >= 10 & Тф < 20')['Фо'].sum() / table1['Фо'].sum(),
    table1.query('Тф >= 20')['Фо'].sum()  / table1['Фо'].sum()
]}, index=[
    'до 5 лет', '5-10 лет', '10-20 лет', 'свыше 20 лет'
])

### Коэффициенты пересчета балансовой стоимости ОФ в восстановительную

In [ ]:
table1_cfs = pd.DataFrame(data={
    'нормы амортизац. отчислений, %': [2.00, 5.00, 5.00, 4.40, 7.00, 14.30, 12.00, 12.00, 25.00, 9.00],
    'коэффициент пересчета': [1.67, 1.71, 1.71, 1.10, 1.45, 1.15, 1.05, 1.35, 1.30, 1.05]
}, index=table1.index)
table1_cfs

### Срок службы и износ

Примем $a$ — норма амортизационных отчислений, $T_{ф}$ — средний фактический срок службы.

#### 4. Нормативный срок службы

$$T_{н} = \frac{1}{a} \cdot 100$$

#### 5. Коэффициент физического износа

$$K_{фи} = \frac{T_{ф}}{T_{н}}$$

#### 6. Износ

$$И_i = \overline{Ф}_{oi} \cdot a \cdot T_{ф} \cdot \frac{1}{100}$$

#### 7. Коэффициент износа

$$K_{изн_i} = \frac{И_i}{\overline{Ф}_{oi}}$$

In [ ]:
standard_service_life = (table1_cfs['нормы амортизац. отчислений, %'] / 100).rename('$T_{н}$')
physical_wear_ratio = (standard_service_life / table1['Тф']).rename('$K_{фи}$')
wear = (table1['Фо'] * table1['Тф'] * table1_cfs['нормы амортизац. отчислений, %'] / 100).rename('$И_i$')
wear_factor = (wear / table1['Фо']).rename('$K_{изн_i}$')

pd.DataFrame(standard_service_life).join(physical_wear_ratio).join(wear).join(wear_factor)

## Оценка основных фондов

### 8. Среднегодовая стоимость ОФ

$$\overline{Ф}_o = \overline{Ф}_o^{\ н} + \frac{\overline{Ф}_o^{\ вв} \cdot m}{12} - \frac{\overline{Ф}_o^{\ выб} \cdot (12 - m)}{12}$$

где:

* $\overline{Ф}_o^{\ н}$ — стоимость ОФ на начало периода, 
* $\overline{Ф}_o^{\ вв}$ — стоимость введенных ОФ,
* $\overline{Ф}_o^{\ выб}$ — стоимость выбывших (ликвидированных) ОФ,
* $m$ — число месяцев функционирования ОФ.

In [ ]:
cost_at_period_start = table3['Показатели']['Стоимость ОПФ на 1 января текущего года, тыс. у.д.е.']
cost_new = table3['Показатели']['Введено ОПФ в текущем году, тыс. у.д.е.']
cost_expired = table3['Показатели']['Выбыло ОПФ в текущем году, тыс. у.д.е.']
months_functioning = 12 - table3['Показатели']['Дата ввода, номер месяца'] + table3['Показатели']['Дата выбытия, номер месяца']
avg_annual_cost = cost_at_period_start + (cost_new * months_functioning / 12) - (cost_expired * (12 - months_functioning) / 12)

f'{np.round(avg_annual_cost, 3)} тыс. у.д.е'

### 9. Стоимость ОФ на конец периода

$$Ф_o^{\ к} = Ф_o^{\ н} + Ф_o^{\ вв} - Ф_o^{\ выб}$$

In [ ]:
closing_cost = cost_at_period_start + cost_new - cost_expired
f'{closing_cost} тыс. у.д.е'

### 10. Коэффициент ввода (обновления)

$$K_{вв} = \frac{Ф_o^{\ вв}}{Ф_o^{\ к}}$$

In [ ]:
renewal_rate = cost_new / closing_cost
np.round(renewal_rate, 3)

### 11. Коэффициент выбытия

$$K_{выб} = \frac{Ф_o^{\ выб}}{Ф_o^{\ н}}$$

In [ ]:
retirement_rate = cost_expired / cost_at_period_start
np.round(retirement_rate, 3)

## Расчет амортизации основных фондов

### 12. Первоначальная стоимость ОФ (балансовая стоимость)

$$Ф_{оi}^{\ б} = Ц_i + С_т + С_м$$

где:

* $Ц_i$ — цена $i$-го вида ОФ,
* $С_т$ — стоимость транспортировки,
* $С_м$ — стоимость монтажа и отладки.

In [ ]:
wholesale_machine_cost = table2['Показатели']['Оптовая цена станка, тыс.руб.']
transportation_cost = wholesale_machine_cost * table2['Показатели']['Транспортные расходы, % от оптовой цены'] / 100
installation_cost = wholesale_machine_cost * table2['Показатели']['Затраты на монтаж и отладку, % от оптовой цены'] / 100
book_cost = wholesale_machine_cost + transportation_cost + installation_cost

f'{np.round(book_cost, 3)} тыс.руб.'

### 13. Восстановительная стоимость

$$Ф_{оi}^{\ в} = Ф_{оi}^{\ б} \cdot K_{пер}$$

где $K_{пер}$ — коэффициент пересчета балансовой стоимости в восстановительную.

In [ ]:
replacement_cost = book_cost * table1_cfs['коэффициент пересчета']['Рабочие машины и оборудование']
f'{np.round(replacement_cost, 3)} тыс.руб.'

### 14. Годовая сумма амортизационных отчислений

$$A_г = \frac{Ф_{оi}^{\ б} \cdot a}{100}$$

In [ ]:
annual_depreciation = book_cost * table1_cfs['нормы амортизац. отчислений, %']['Рабочие машины и оборудование'] / 100
f'{np.round(annual_depreciation, 3)} тыс.руб.'

### 15. Остаточная стоимость

$$Ф_{оi}^{\ о} = Ф_{оi}^{\ б} - A_г \cdot Т_ф$$

In [ ]:
residual_value = book_cost - annual_depreciation * table1['Тф']['Рабочие машины и оборудование']
f'{np.round(residual_value, 3)} тыс.руб.'

## Расчет показателей использования основных фондов

### 16. Фондоотдача

$$f_о = \frac{Q_в}{\overline{Ф}_о}$$

где:

* $Q_в$ — стоимость валовой продукции,
* $\overline{Ф}_о$ — среднегодовая стоимость ОФ.

### 17. Фондовооруженность

$$f_в = \frac{\overline{Ф}_о}{\overline{Ч}_{пр}}$$

где $\overline{Ч}_{пр}$ — среднегодовая численность производственных рабочих.

### 18. Индекс изменения показателя

$$J_{f_о} = \frac{f_о^о}{f_о^n},\ J_{f_d} = \frac{f_в^о}{f_в^в}$$

где:

* $f_о^о,\ f_в^о$ — показатели фондоотдачи и фондовооруженности в отчетном периоде,
* $f_о^n,\ f_в^n$ — в плановом периоде.

### 19. Индекс изменения производительности труда

$$J_{nm} = J_{f_о} \cdot J_{f_в}$$

### 20. Коэффициент экстенсивного использования оборудования

$$K_э = \frac{Ф_ф}{Ф_э}$$

где:

* $Ф_э$ — эффективный фонд времени работы установленного оборудования за месяц, станко-час,
* $Ф_ф$ — фактически отработано оборудованием за месяц, станко-час.

### 21. Эффективный фонд времени единицы работы оборудования за месяц

$$Ф_э = Д \cdot С \cdot f \cdot (1 - \frac{П_р}{100}) \cdot n$$

где:

* $Д$ — число рабочих дней в месяце (22 дня),
* $С$ — сменность (2 смены),
* $f$ — продолжительность рабочей смены (8 часов),
* $П_р$ — затраты времени на планово-предупредительные работы (3%),
* $n$ — число единиц оборудования.

### 22. Коэффициент интенсивного использования оборудования

$$K_u = \frac{В_ф}{В_n},\ В_n = П_ч \cdot f \cdot n$$

где:

* $В_ф$ — фактическая производительность за смену, штук,
* $В_n$ — плановая производительность за смену, штук,
* $П_ч$ — часовая производительность единицы оборудования.

### 23. Коэффициент интегрального использования

$$K_{un} = K_э \cdot К_u$$

### 24. Коэффициент сменности

$$K_{см} = \frac{Ф_ф}{Ф'_э},\ Ф'_э = Д \cdot f \cdot n$$

где $Ф'_э$ — максимальное число станко-часов, которые может быть отработано при односменной работе.